
# SRAG EDA – Bronze to Silver Preview

This notebook provides a clean **exploratory data analysis (EDA)** for the SRAG dataset loaded into DuckDB.  
It uses robust path resolution, safe connections, and reusable helpers to:
- Inspect **bronze** (`raw_all`) structure and sizes;
- Preview a **silver-like** transformed selection (dates parsed, categories normalized);
- Produce essential distributions for mortality, ICU, vaccination, age, and UF;

In [3]:

"""Utility functions for path resolution and DuckDB connection."""
from __future__ import annotations

from pathlib import Path
from typing import Dict, Iterable, Optional

import os
import duckdb
import pandas as pd


def find_project_root(marker: str = "pyproject.toml") -> Path:
    """Return the nearest ancestor directory that contains ``marker``.

    Args:
        marker: File name used to detect the project root (default: pyproject.toml).

    Returns:
        The project root path. If not found, returns the current working directory.
    """
    here = Path.cwd().resolve()
    for p in (here, *here.parents):
        if (p / marker).exists():
            return p
    return here


def connect_duckdb(db_path: Path, read_only: bool = True) -> duckdb.DuckDBPyConnection:
    """Connect to DuckDB and set the active schema to 'srag'.

    Args:
        db_path: Path to the DuckDB database file.
        read_only: Whether to open the connection in read-only mode.

    Returns:
        A DuckDB connection with the schema set to 'srag'.

    Raises:
        FileNotFoundError: If the DuckDB file does not exist.
    """
    if read_only and not db_path.exists():
        raise FileNotFoundError(
            f"DuckDB not found at {db_path}. Run the ETL (e.g., 'python run_etl.py')."
        )
    con = duckdb.connect(str(db_path), read_only=read_only)
    con.execute("SET schema 'srag';")
    return con


def run_sql_df(con: duckdb.DuckDBPyConnection, sql: str, params: Optional[list] = None) -> pd.DataFrame:
    """Run a SQL statement and return a pandas DataFrame.

    Args:
        con: DuckDB connection.
        sql: SQL query string.
        params: Optional list of parameters to bind to the query.

    Returns:
        Query result as a pandas DataFrame.
    """
    return con.execute(sql, params or []).df()


def existing_columns(con: duckdb.DuckDBPyConnection, table: str) -> Dict[str, str]:
    """Return a map of lowercased column name -> exact column name for a table.

    Args:
        con: DuckDB connection.
        table: Table name within the current schema (e.g., 'raw_all').

    Returns:
        Dictionary mapping lowercased column names to their exact names.
    """
    rows = con.execute(
        """
        SELECT column_name
        FROM information_schema.columns
        WHERE table_schema = current_schema()
          AND table_name = ?
        """, [table],
    ).fetchall()
    return {r[0].lower(): r[0] for r in rows}


def first_existing(cols_map: Dict[str, str], candidates: Iterable[str]) -> Optional[str]:
    """Return the first exact column name that exists among ``candidates``.

    Args:
        cols_map: Mapping from lowercased column names to exact names.
        candidates: Iterable of candidate column names to search for.

    Returns:
        The exact column name if found, otherwise ``None``.
    """
    for cand in candidates:
        hit = cols_map.get(cand.lower())
        if hit:
            return hit
    return None


In [4]:

"""Connect to DuckDB (read-only) using project-relative path."""
ROOT = find_project_root()
DB_PATH = ROOT / "data" / "srag.duckdb"

# Export to env for any helper code that relies on DUCKDB_PATH.
os.environ.setdefault("DUCKDB_PATH", str(DB_PATH))

con = connect_duckdb(DB_PATH, read_only=True)
print("Project root:", ROOT)
print("DuckDB path :", DB_PATH)
run_sql_df(con, "SHOW TABLES;")


Project root: /home/ziolli/Área de trabalho/Case-Indicium
DuckDB path : /home/ziolli/Área de trabalho/Case-Indicium/data/srag.duckdb


,name
0,raw_2019
1,raw_2020
2,raw_2021
3,raw_2022
4,raw_2023
5,raw_2024
6,raw_2025
7,raw_all


## Inventory: structure and sizes

In [5]:

# Table schema
run_sql_df(con, "PRAGMA table_info('raw_all')")


,cid,name,type,notnull,dflt_value,pk
0,0,year,INTEGER,False,None,False
1,1,source_url,VARCHAR,False,None,False
2,2,ingested_at,TIMESTAMP,False,None,False
3,3,NU_NOTIFIC,BIGINT,False,None,False
4,4,DT_NOTIFIC,DATE,False,None,False
...,...,...,...,...,...,...
192,192,VG_DTRES,VARCHAR,False,None,False
193,193,VG_ENC,VARCHAR,False,None,False
194,194,VG_REINF,VARCHAR,False,None,False
195,195,VG_CODEST,VARCHAR,False,None,False


In [6]:

# Row counts by year
run_sql_df(con, """
SELECT year, COUNT(*) AS rows
FROM raw_all
GROUP BY 1
ORDER BY 1
""")


,year,rows
0,2019,48961
1,2020,1206920
2,2021,1745672
3,2022,560577
4,2023,279453
5,2024,267984
6,2025,245358


In [8]:

# Sample a few records
run_sql_df(con, "SELECT * FROM raw_all LIMIT 10")


,year,source_url,ingested_at,NU_NOTIFIC,DT_NOTIFIC,SEM_NOT,DT_SIN_PRI,SEM_PRI,SG_UF_NOT,ID_REGIONA,...,VG_OMS,VG_OMSOUT,VG_LIN,VG_MET,VG_METOUT,VG_DTRES,VG_ENC,VG_REINF,VG_CODEST,REINF
0,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478195042,2019-01-10,02,2019-01-06,02,MG,BELO HORIZONTE,...,None,None,None,None,None,None,None,None,None,<NA>
1,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478195276,2019-01-03,01,2019-01-01,01,SP,GVE I CAPITAL,...,None,None,None,None,None,None,None,None,None,<NA>
2,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478207219,2019-01-02,01,2018-12-31,01,PE,001,...,None,None,None,None,None,None,None,None,None,<NA>
3,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478211086,2019-01-10,02,2019-01-07,02,SP,GVE XVII CAMPINAS,...,None,None,None,None,None,None,None,None,None,<NA>
4,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478212765,2019-01-11,02,2019-01-06,02,PE,004,...,None,None,None,None,None,None,None,None,None,<NA>
5,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478215714,2019-01-02,01,2018-12-30,01,PE,001,...,None,None,None,None,None,None,None,None,None,<NA>
6,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478218037,2019-01-08,02,2019-01-02,01,MS,EX ARS DE CAMPO GRANDE,...,None,None,None,None,None,None,None,None,None,<NA>
7,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478218301,2019-01-03,01,2019-01-01,01,PE,001,...,None,None,None,None,None,None,None,None,None,<NA>
8,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478220832,2019-01-03,01,2019-01-02,01,PE,001,...,None,None,None,None,None,None,None,None,None,<NA>
9,2019,https://s3.sa-east-1.amazonaws.com/ckan.saude....,2025-09-14 19:36:42.428,315478224482,2019-01-03,01,2019-01-01,01,PE,001,...,None,None,None,None,None,None,None,None,None,<NA>


## Basic quality checks: null ratios and categories

In [9]:

cols = existing_columns(con, "raw_all")
c_evol = first_existing(cols, ["EVOLUCAO"])
c_uti  = first_existing(cols, ["UTI"])
c_vac  = first_existing(cols, ["VACINA_COV"])
c_uf   = first_existing(cols, ["SG_UF_NOT"])
c_sexo = first_existing(cols, ["CS_SEXO"])
c_idad = first_existing(cols, ["NU_IDADE_N"])

summary_sql = f"""
SELECT
  COUNT(*) AS total_rows
  {", SUM(" + c_evol + " IS NULL) AS null_evolucao" if c_evol else ""}
  {", SUM(" + c_uti  + " IS NULL) AS null_uti"      if c_uti  else ""}
  {", SUM(" + c_vac  + " IS NULL) AS null_vacina"   if c_vac  else ""}
  {", SUM(" + c_uf   + " IS NULL) AS null_uf"       if c_uf   else ""}
  {", SUM(" + c_sexo + " IS NULL) AS null_sexo"     if c_sexo else ""}
  {", SUM(" + c_idad + " IS NULL) AS null_idade"    if c_idad else ""}
FROM raw_all
"""

run_sql_df(con, summary_sql)


,total_rows,null_evolucao,null_uti,null_vacina,null_uf,null_sexo,null_idade
0,4354925,327517.0,574060.0,1531252.0,10581.0,0.0,0.0


In [10]:

# Category distributions
out = {}
if c_evol:
    out["evolucao"] = run_sql_df(con, f"""
        SELECT {c_evol} AS evolucao, COUNT(*) AS n
        FROM raw_all
        GROUP BY 1 ORDER BY n DESC
    """)
if c_uti:
    out["uti"] = run_sql_df(con, f"""
        SELECT {c_uti} AS uti, COUNT(*) AS n
        FROM raw_all
        GROUP BY 1 ORDER BY n DESC
    """)
if c_vac:
    out["vacina_cov"] = run_sql_df(con, f"""
        SELECT {c_vac} AS vacina_cov, COUNT(*) AS n
        FROM raw_all
        GROUP BY 1 ORDER BY n DESC
    """)
if c_sexo:
    out["sexo"] = run_sql_df(con, f"""
        SELECT {c_sexo} AS sexo, COUNT(*) AS n
        FROM raw_all
        GROUP BY 1 ORDER BY n DESC
    """)
if c_uf:
    out["uf"] = run_sql_df(con, f"""
        SELECT {c_uf} AS uf, COUNT(*) AS n
        FROM raw_all
        GROUP BY 1 ORDER BY n DESC
    """)

out  # Display dict of DataFrames


{'evolucao':    evolucao        n
 0       1.0  2932434
 1       2.0   918614
 2       NaN   327517
 3       9.0    98579
 4       3.0    77781,
 'uti':     uti        n
 0     2  2463199
 1     1  1238896
 2  <NA>   574060
 3     9    78770,
 'vacina_cov':    vacina_cov        n
 0        <NA>  1531252
 1           2  1230280
 2           1  1226107
 3           9   367286,
 'sexo':   sexo        n
 0    M  2320504
 1    F  2033631
 2    I      790,
 'uf':       uf        n
 0     SP  1234237
 1     MG   481641
 2     RJ   363866
 3     PR   350664
 4     RS   237425
 5     CE   186576
 6     SC   170463
 7     PE   169526
 8     BA   163326
 9     GO   128660
 10    DF   110293
 11    PA    96021
 12    MS    87058
 13    AM    72717
 14    PB    66343
 15    MT    65528
 16    MA    50630
 17    SE    45177
 18    ES    42724
 19    AL    41237
 20    PI    40347
 21    RN    40144
 22    RO    30088
 23    TO    28017
 24    AC    17601
 25    AP    14998
 26  None    10581
 27    

## Date ranges (min/max)

In [11]:

# Checking Range
c_dt_not = first_existing(cols, ["DT_NOTIFIC", "dt_notific"])
c_dt_sin = first_existing(cols, ["DT_SIN_PRI", "dt_sin_pri"])

if c_dt_not and c_dt_sin:
    run_sql_df(con, f"""
        SELECT
          MIN(TRY_STRPTIME(CAST({c_dt_not} AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d'])) AS min_notif,
          MAX(TRY_STRPTIME(CAST({c_dt_not} AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d'])) AS max_notif,
          MIN(TRY_STRPTIME(CAST({c_dt_sin} AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d'])) AS min_sintoma,
          MAX(TRY_STRPTIME(CAST({c_dt_sin} AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d'])) AS max_sintoma
        FROM raw_all
    """)
else:
    pd.DataFrame(
        {"warning": ["DT_NOTIFIC / DT_SIN_PRI not found in raw_all"]}
    )


## Silver-like preview

In [12]:

silver_preview_sql = """
WITH parsed AS (
  SELECT
    COALESCE(
      TRY_STRPTIME(CAST(DT_NOTIFIC AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d']),
      TRY_CAST(DT_NOTIFIC AS DATE)
    ) AS dt_notific,
    COALESCE(
      TRY_STRPTIME(CAST(DT_SIN_PRI AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d']),
      TRY_CAST(DT_SIN_PRI AS DATE)
    ) AS dt_sin_pri,
    COALESCE(
      TRY_STRPTIME(CAST(DT_EVOLUCA AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d']),
      TRY_CAST(DT_EVOLUCA AS DATE)
    ) AS dt_evoluca,
    COALESCE(
      TRY_STRPTIME(CAST(DT_ENCERRA AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d']),
      TRY_CAST(DT_ENCERRA AS DATE)
    ) AS dt_encerra,

    TRY_CAST(SEM_NOT AS INTEGER)     AS sem_not,
    TRY_CAST(EVOLUCAO AS INTEGER)    AS evolucao_code,
    TRY_CAST(CLASSI_FIN AS INTEGER)  AS classi_fin,
    TRY_CAST(UTI AS INTEGER)         AS uti_code,

    COALESCE(
      TRY_STRPTIME(CAST(DT_ENTUTI AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d']),
      TRY_CAST(DT_ENTUTI AS DATE)
    ) AS dt_entuti,
    COALESCE(
      TRY_STRPTIME(CAST(DT_SAIDUTI AS VARCHAR), ['%d/%m/%Y','%Y-%m-%d']),
      TRY_CAST(DT_SAIDUTI AS DATE)
    ) AS dt_saiduti,

    TRY_CAST(VACINA_COV AS INTEGER)  AS vacina_cov_code,
    TRY_CAST(NU_IDADE_N AS INTEGER)  AS idade,
    UPPER(TRIM(CS_SEXO))             AS sexo,
    UPPER(TRIM(SG_UF_NOT))           AS uf
  FROM raw_all
)
SELECT
  dt_notific,
  dt_sin_pri,
  dt_evoluca,
  dt_encerra,
  DATE_TRUNC('month', dt_notific)              AS mes_notific,
  EXTRACT('year' FROM dt_notific)::INT         AS ano_notific,
  sem_not,
  CASE evolucao_code
    WHEN 1 THEN 'CURA'
    WHEN 2 THEN 'OBITO'
    WHEN 3 THEN 'OBITO_OUTRAS'
    WHEN 9 THEN 'IGNORADO'
    ELSE NULL
  END AS evolucao_label,
  evolucao_code,
  classi_fin,
  CASE
    WHEN uti_code = 1 THEN TRUE
    WHEN uti_code = 2 THEN FALSE
    ELSE NULL
  END AS uti_bool,
  dt_entuti,
  dt_saiduti,
  CASE
    WHEN vacina_cov_code = 1 THEN TRUE
    WHEN vacina_cov_code = 2 THEN FALSE
    ELSE NULL
  END AS vacinado_bool,
  idade,
  CASE
    WHEN idade IS NULL THEN NULL
    WHEN idade < 5 THEN '0-4'
    WHEN idade BETWEEN 5 AND 17 THEN '5-17'
    WHEN idade BETWEEN 18 AND 39 THEN '18-39'
    WHEN idade BETWEEN 40 AND 59 THEN '40-59'
    ELSE '60+'
  END AS faixa_etaria,
  sexo,
  uf,
  CASE WHEN evolucao_code = 2 THEN TRUE ELSE FALSE END AS is_obito,
  CASE
    WHEN dt_notific <= CURRENT_DATE - INTERVAL 60 DAY
         AND (evolucao_code IS NULL OR evolucao_code = 9 OR dt_encerra IS NULL)
    THEN TRUE ELSE FALSE
  END AS pendente_60d
FROM parsed
LIMIT 100;
"""

run_sql_df(con, silver_preview_sql)


,dt_notific,dt_sin_pri,dt_evoluca,dt_encerra,mes_notific,ano_notific,sem_not,evolucao_label,evolucao_code,classi_fin,uti_bool,dt_entuti,dt_saiduti,vacinado_bool,idade,faixa_etaria,sexo,uf,is_obito,pendente_60d
0,2019-01-10,2019-01-06,2019-01-25,2019-02-15,2019-01-01,2019,2,CURA,1,4,False,NaT,NaT,<NA>,30,18-39,M,MG,False,False
1,2019-01-03,2019-01-01,2019-03-05,2019-01-21,2019-01-01,2019,1,CURA,1,4,True,2019-01-01,NaT,<NA>,7,5-17,F,SP,False,False
2,2019-01-02,2018-12-31,2019-02-12,2019-05-31,2019-01-01,2019,1,CURA,1,4,False,NaT,NaT,<NA>,1,0-4,M,PE,False,False
3,2019-01-10,2019-01-07,2019-01-29,2019-02-04,2019-01-01,2019,2,CURA,1,2,True,2019-01-10,NaT,<NA>,5,5-17,F,SP,False,False
4,2019-01-11,2019-01-06,2019-01-13,2019-05-21,2019-01-01,2019,2,OBITO,2,4,<NA>,NaT,NaT,<NA>,3,0-4,F,PE,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2019-01-10,2019-01-08,2019-01-10,2019-01-28,2019-01-01,2019,2,OBITO,2,4,True,2019-01-10,2019-01-10,<NA>,57,40-59,M,PR,True,False
96,2019-01-04,2019-01-03,2019-01-15,2019-01-24,2019-01-01,2019,1,CURA,1,4,True,2019-01-03,2019-01-11,<NA>,1,0-4,F,PR,False,False
97,2019-01-03,2018-12-30,2019-01-15,2019-01-28,2019-01-01,2019,1,CURA,1,4,True,2019-01-03,NaT,<NA>,59,40-59,M,SP,False,False
98,2019-01-17,2019-01-09,2019-02-08,2019-02-21,2019-01-01,2019,3,CURA,1,2,True,2019-01-16,2019-01-31,<NA>,25,18-39,F,MG,False,False


| Column           | Type    | Source / Derivation                                                                        | Description                                                | Notes / Allowed values                                         |
| ---------------- | ------- | ------------------------------------------------------------------------------------------ | ---------------------------------------------------------- | -------------------------------------------------------------- |
| `dt_notific`     | DATE    | Parsed from `DT_NOTIFIC`             | Notification date of the case.                             | Used as primary time axis.                                     |
| `dt_sin_pri`     | DATE    | Parsed from `DT_SIN_PRI`                                                                   | Symptom onset date.                                        | Useful for epi curves by onset.                                |
| `dt_evoluca`     | DATE    | Parsed from `DT_EVOLUCA`                                                                   | Outcome date (discharge or death).                         | May be NULL (depends on outcome).                              |
| `dt_encerra`     | DATE    | Parsed from `DT_ENCERRA`                                                                   | Case closure date in the system.                           | Key to detect “closed” vs “pending”.                           |
| `mes_notific`    | DATE    | `DATE_TRUNC('month', dt_notific)`                                                          | Month bucket (first day of month).                         | For monthly aggregates.                                        |
| `ano_notific`    | INTEGER | `EXTRACT('year' FROM dt_notific)`                                                          | Notification year.                                         | —                                                              |
| `sem_not`        | INTEGER | `TRY_CAST(SEM_NOT AS INTEGER)`                                                             | Epidemiological week (of notification).                    | Depends on original CSV field.                                 |
| `evolucao_label` | VARCHAR | Mapped from `evolucao_code` (1→CURA, 2→OBITO, 3→OBITO\_OUTRAS, 9→IGNORADO)                 | Human-readable outcome label.                              | NULL if unmapped.                                              |
| `evolucao_code`  | INTEGER | `TRY_CAST(EVOLUCAO AS INTEGER)`                                                            | Outcome code.                                              | {1=CURA, 2=ÓBITO, 3=ÓBITO\_OUTRAS, 9=IGNORADO}.                |
| `classi_fin`     | INTEGER | `TRY_CAST(CLASSI_FIN AS INTEGER)`                                                          | Final case classification (disease/etiology code).         | Label mapping can be applied later.                            |
| `uti_bool`       | BOOLEAN | From `UTI` (1→TRUE, 2→FALSE, else NULL)                                                    | Whether patient had ICU admission.                         | Not occupancy; just “case had ICU”.                            |
| `dt_entuti`      | DATE    | Parsed from `DT_ENTUTI`                                                                    | ICU entry date.                                            | May be NULL.                                                   |
| `dt_saiduti`     | DATE    | Parsed from `DT_SAIDUTI`                                                                   | ICU discharge date.                                        | May be NULL.                                                   |
| `vacinado_bool`  | BOOLEAN | From `VACINA_COV` (1→TRUE, 2→FALSE, else NULL)                                             | Whether patient had prior COVID vaccination.               | Does not measure population coverage.                          |
| `idade`          | INTEGER | `TRY_CAST(NU_IDADE_N AS INTEGER)`                                                          | Age in years.                                              | May contain extreme/invalid values; validate if needed.        |
| `faixa_etaria`   | VARCHAR | Derived from `idade` (0–4, 5–17, 18–39, 40–59, 60+)                                        | Age band for stratification.                               | NULL if `idade` is NULL.                                       |
| `sexo`           | VARCHAR | `UPPER(TRIM(CS_SEXO))`                                                                     | Sex.                                                       | Common: {M, F, I, …}.                                          |
| `uf`             | VARCHAR | `UPPER(TRIM(SG_UF_NOT))`                                                                   | State of notification.                                     | 2-letter UF code.                                              |
| `is_obito`       | BOOLEAN | `evolucao_code = 2`                                                                        | Case with death outcome.                                   | For mortality rate denominators, prefer only “closed” cohorts. |
| `pendente_60d`   | BOOLEAN | `dt_notific <= CURRENT_DATE - 60d AND (evolucao_code IS NULL OR =9 OR dt_encerra IS NULL)` | Likely pending case after 60 days without outcome/closure. | Tweak the 60-day window if needed.                             |
